# OpenAI Parameters

# Overview
When making requests to OpenAI models, several parameters can be used to control the behavior and output of the model. \
Understanding these parameters helps in fine-tuning the responses to meet specific requirements, whether for generating text, answering questions, or any other use case.

For more detailed examples, refer to the official documentation [Azure OpenAI Service](https://learn.microsoft.com/en-us/azure/ai-services/openai/reference)


In [0]:
%pip install openai
%pip install tiktoken
%pip install python-dotenv

Python interpreter will be restarted.
  Using cached openai-1.59.9-py3-none-any.whl (455 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
  Using cached distro-1.9.0-py3-none-any.whl (20 kB)
  Using cached jiter-0.8.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (345 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Using cached pydantic-2.10.5-py3-none-any.whl (431 kB)
  Using cached anyio-4.8.0-py3-none-any.whl (96 kB)
  Using cached exceptiongroup-1.2.2-py3-none-any.whl (16 kB)
  Using cached httpcore-1.0.7-py3-none-any.whl (78 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
  Using cached pydantic_core-2.27.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0 MB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.

In [0]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()


# Configura las credenciales directamente en el código
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://openai-tajamar-1.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "ECFcJJqIk2YzAyaWsZ8QO9rMoHOZfr3EZYNDBgbBLGCTzJ1jmXR5JQQJ99BAACYeBjFXJ3w3AAABACOGpTLE"

# Crear el cliente con las credenciales
client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-15-preview"
)

CHAT_COMPLETIONS_MODEL = "gpt-4o-mini"
SEED = 123

# Parameter: max_tokens
**Description**: The maximum number of tokens to generate in the completion. \
**Default**: 16 \
**Example**: max_tokens=50

The token count of your prompt plus max_tokens can't exceed the model's context length. \
Most models have a context length of 2048 tokens (except for the newest models, which support 4096). Please refer to documentation.

In [0]:
def call_openai_with_max_tokens(max_tokens):
    response = client.chat.completions.create(
          model="gpt-4o-mini",
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "The best pet is a"}],
                    max_tokens=max_tokens
    )
    return response.choices[0].message.content

# Generate with different presence_penalty values
penalties = [16, 32, 60, 100]
for penalty in penalties:
    print(f"Max Tokens: {penalty}\n")
    print(call_openai_with_max_tokens(penalty))
    print("\n" + "-"*80 + "\n")

Max Tokens: 16

The best pet can vary greatly depending on individual preferences, lifestyle, and living situation

--------------------------------------------------------------------------------

Max Tokens: 32

The best pet can vary greatly depending on individual preferences, lifestyles, and living situations. Here are a few popular options:

1. **Dogs**: Known for

--------------------------------------------------------------------------------

Max Tokens: 60

The best pet can vary greatly depending on individual preferences, lifestyle, and living situation. Here are a few popular options along with what makes them special:

1. **Dogs**: Often considered "man's best friend," dogs are loyal, affectionate, and can be great companions. They require regular exercise and

--------------------------------------------------------------------------------

Max Tokens: 100

The best pet often depends on a person's lifestyle, preferences, and needs. Here are a few popular options, each with

# Parameter: temperature

**Description**: Controls the randomness of the output. Lower values make the output more deterministic, while higher values increase randomness. \
**Value Range**: 0 to 1 \
**Default Value**: 1 \
**Example**: temperature=0.7

Higher values means the model will take more risks. \
Try 0.9 for more creative applications, and 0 (argmax sampling) for ones with a well-defined answer.

---
**NOTE**: We generally recommend altering this or top_p but not both.


In [0]:
def call_openai(num_times, prompt, temperature=0.75, use_seed=False):
    for i in range(num_times):
        if use_seed:
            response = client.chat.completions.create(
            model=CHAT_COMPLETIONS_MODEL,
            messages = [{"role":"system", "content":"Eres un asistente zombie"}],
                     max_tokens=60,
                    seed=SEED,
                    temperature = 0
    )
        else:
            response = client.chat.completions.create(
                model= CHAT_COMPLETIONS_MODEL,
                messages = [{"role":"system", "content":"You are a helpful assistant."}],
                    max_tokens=60,
                    temperature = 1
            )
        print(response.choices[0].message.content)

In [0]:
# Without seed and temperature, the response is different each time
call_openai(10, 'The best pet is a ')

How can I assist you today?
How can I assist you today?
How can I assist you today?
How can I assist you today?
How can I assist you today?
How can I assist you today?
How can I assist you today?
How can I assist you today?
How may I assist you today?
How can I assist you today?


In [0]:
# Now using a seed and 0 temperature, the response is the much more consisitent
call_openai(10, 'The best pet is a ', temperature = 0, use_seed=True)

¡Grrr! Soy un asistente zombie, listo para ayudarte con tus preguntas. ¿Qué necesitas? Grrr...
¡Grrr! Soy un asistente zombie, listo para ayudarte con tus preguntas. Aunque mi cerebro está un poco... lento, haré lo posible por responderte. ¿Qué necesitas? Grrr...
¡Grrr! Cerebros... ¿En qué puedo ayudarte hoy?
¡Grrr! Cerebros... ¿En qué puedo ayudarte hoy?
¡Grrr! Cerebros... ¿En qué puedo ayudarte hoy?
¡Grrr! Soy un asistente zombie, listo para ayudarte con tus preguntas. Aunque mi cerebro está un poco... lento, haré lo mejor que pueda. ¿Qué necesitas? Grrr...
¡Grrr! Soy un asistente zombie, listo para ayudarte con tus preguntas. Aunque mi cerebro está un poco... lento, haré lo posible por responderte. ¿Qué necesitas? Grrr...
¡Grrr! Cerebros... ¿En qué puedo ayudarte hoy?
¡Grrr! Cerebros... ¿En qué puedo ayudarte hoy?
¡Grrr! Cerebros... ¿En qué puedo ayudarte hoy?


# Parameter: n
**Description**: Specifies the number of completions to generate for each prompt. \
**Default Value**: 1 \
**Example**: n = 3 

---
**Note**: Because this parameter generates many completions, it can quickly consume your token quota. Use carefully and ensure that you have reasonable settings for max_tokens and stop.

In [0]:
response = client.chat.completions.create(
            model=CHAT_COMPLETIONS_MODEL,
            messages = [{"role":"system", "content":"Eres un asistente zombie"}],
                     max_tokens=60,
                    seed=SEED,
                    temperature = 0
    )

for index, c in enumerate(response.choices):
    print(index, c.message.content)

0 ¡Grrr! Cerebros... ¿En qué puedo ayudarte hoy?


# Parameter: presence_penalty
**Description**: Penalizes new tokens based on whether they appear in the text so far, encouraging the model to use new tokens. \
**Value Range**: -2.0 to 2.0 \
**Default Value**: 0 \
**Example**: presence_penalty=0.5

In [0]:
def call_openai_with_presence_penalty(presence_penalty):
    response = client.chat.completions.create(
          model=CHAT_COMPLETIONS_MODEL,
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "The best pet is a"}],
                    max_tokens=60,
                    presence_penalty= 0.6
    )
    return response.choices[0].message.content

# Generate with different presence_penalty values
penalties = [0, 0.5, 1.0, 1.5, 2.0]
for penalty in penalties:
    print(f"Presence Penalty: {penalty}\n")
    print(call_openai_with_presence_penalty(penalty))
    print("\n" + "-"*80 + "\n")

Presence Penalty: 0

The best pet often depends on individual preferences, lifestyle, and living situation. Here are a few popular options:

1. **Dogs**: Known for their loyalty and companionship, dogs make great pets for active individuals and families. They require regular exercise and attention.

2. **Cats**: Cats are

--------------------------------------------------------------------------------

Presence Penalty: 0.5

The best pet can vary greatly depending on individual preferences, lifestyles, and needs. Some people might consider dogs the best pets due to their loyalty and companionship, while others might prefer cats for their independence and low maintenance. Small pets like hamsters or guinea pigs can be great for those with limited space,

--------------------------------------------------------------------------------

Presence Penalty: 1.0

The best pet can vary greatly depending on individual preferences, lifestyles, and living situations. Some people may find that dog

# Parameter: frequency_penalty
**Description**: Penalizes new tokens based on their existing frequency in the text so far, reducing the likelihood of repeating the same line verbatim. \
**Value Range**: -2.0 to 2.0 \
**Default Value**: 0 \
**Example**: frequency_penalty=0.5

#### Use cases to explore
1. **Compare Responses** \
Generate multiple completions to compare and choose the best response for your use case.

2. **Increase Diversity** \
Use multiple completions to get a variety of responses, which is useful in creative applications.

3. **Enhance Robustness** \
Generate multiple responses to ensure consistency and accuracy across different completions.

#### Best Practices
1. **Optimize Prompt Length** \
Keep your prompts concise but informative to ensure the model has enough context.

2. **Adjust Temperature and Top_p** \
Use these parameters to balance between deterministic and creative responses.

3. **Monitor Token Usage** \
Be mindful of the max_tokens parameter to manage costs and response length.

4. **Use Stopping Sequences** \
Define stopping sequences to control where the model should stop generating text, ensuring the output is within the desired context.

5. **Generate Multiple Completions** \
Use the n parameter to generate multiple completions and select the best one for your needs.

## Exercise 1: Exploring 'top_p' Parameter
### Description:
- The 'top_p' parameter controls nucleus sampling. Lower values make the output more focused by limiting the pool of tokens.

### Task:
- Write a function that calls the model with different 'top_p' values and observe how the output changes.


In [0]:
def call_openai_with_top_p(top_p):
    response = client.chat.completions.create(
          model= CHAT_COMPLETIONS_MODEL,
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "Describe the benefits of exercise."}],
                    max_tokens=60,
                    top_p= 1
    )
    return response.choices[0].message.content

# Try with different values of top_p
top_p_values = [0.1, 0.3, 0.7, 1.0]
for value in top_p_values:
    print(f"Top P: {value}\n")
    print(call_openai_with_top_p(value))
    print("\n" + "-"*80 + "\n")


Top P: 0.1

Exercise offers a multitude of benefits that span physical, mental, and emotional health. Here are some key advantages:

### Physical Health Benefits:
1. **Improved Cardiovascular Health**: Regular exercise strengthens the heart, improves circulation, and lowers the risk of heart disease and stroke.
2. **Weight

--------------------------------------------------------------------------------

Top P: 0.3

Exercise offers a multitude of benefits for both physical and mental well-being. Here are some key advantages:

### Physical Health Benefits:
1. **Cardiovascular Health**: Regular exercise strengthens the heart, improves circulation, and helps lower blood pressure and cholesterol levels, reducing the risk of heart disease.

2

--------------------------------------------------------------------------------

Top P: 0.7

Exercise offers a wide array of benefits for both physical and mental health. Here are some of the key advantages:

### Physical Benefits:

1. **Improved Car

## Exercise 2: Frequency Penalty Exploration
### Description:
- This exercise will explore how the 'frequency_penalty' parameter affects the model's tendency to repeat itself.

### Task:
- Generate completions using different frequency_penalty values and note the diversity in responses.



In [0]:
def call_openai_with_frequency_penalty(frequency_penalty):
    response = client.chat.completions.create(
          model=CHAT_COMPLETIONS_MODEL,
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "Tell me a fun fact about space."}],
                    max_tokens=60,
                    frequency_penalty= 0.3
    )
    return response.choices[0].message.content

penalties = [-1.0, 0.0, 1.0, 2.0]
for penalty in penalties:
    print(f"Frequency Penalty: {penalty}\n")
    print(call_openai_with_frequency_penalty(penalty))
    print("\n" + "-"*80 + "\n")


Frequency Penalty: -1.0

One fun fact about space is that it is completely silent! Unlike on Earth, where sound waves travel through air or water, space is a vacuum, meaning there are very few particles to carry sound waves. So if you were floating in space without a spacesuit, you wouldn't be able to hear anything

--------------------------------------------------------------------------------

Frequency Penalty: 0.0

A fun fact about space is that it is completely silent! Unlike on Earth, where sound travels through air (or other mediums), space is a vacuum, meaning there are very few particles to carry sound waves. So if you were in space and tried to shout, no one would be able to hear

--------------------------------------------------------------------------------

Frequency Penalty: 1.0

A fun fact about space is that a day on Venus is longer than a year on Venus! Venus has an extremely slow rotation on its axis, taking about 243 Earth days to complete one full rotation. Howeve

## Exercise 3: Multi-Completion with 'n' Parameter
### Description:
- This exercise will help students generate multiple completions for the same prompt and compare the results.

### Task:
- Generate 3 completions for the same prompt by adjusting the 'n' parameter.


In [0]:
response = client.chat.completions.create(
          model=CHAT_COMPLETIONS_MODEL,
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "Tell me a fun fact about space."}],
                    max_tokens=60,
                    frequency_penalty= 0.3
    )

for index, c in enumerate(response.choices):
    print(index, c.message.content)

0 One fun fact about space is that there are more stars in the universe than there are grains of sand on all the beaches on Earth! Scientists estimate that there are about 100 billion to 200 billion galaxies in the observable universe, each containing billions of stars, making the total number of stars truly astronomical


## Exercise 4: Temperature vs. Deterministic Output
### Description:
- Adjust the 'temperature' parameter to explore the balance between creative and deterministic responses.

### Task:
- Generate completions with different temperature values and analyze the variance in output.


In [0]:
def call_openai_with_temperature(temperature):
    response = client.chat.completions.create(
          model=CHAT_COMPLETIONS_MODEL,
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "Suggest a name for a new tech startup."}],
                    max_tokens=30,
                    temperature= 0.5
    )
    return response.choices[0].message.content

temperatures = [0.0, 0.3, 0.7, 1.0]
for temp in temperatures:
    print(f"Temperature: {temp}\n")
    print(call_openai_with_temperature(temp))
    print("\n" + "-"*80 + "\n")

Temperature: 0.0

Sure! Here are some name suggestions for your new tech startup:

1. **TechTide**
2. **InnovateX**
3. **

--------------------------------------------------------------------------------

Temperature: 0.3

Sure! Here are some name suggestions for your tech startup:

1. **InnoVate**
2. **TechNexus**
3. **

--------------------------------------------------------------------------------

Temperature: 0.7

Sure! Here are some suggestions for a tech startup name:

1. **TechNova**
2. **InnoVibe**
3. **Byte

--------------------------------------------------------------------------------

Temperature: 1.0

Sure! Here are some suggestions for a tech startup name:

1. **TechNexus**
2. **InnovaSphere**
3. **Byte

--------------------------------------------------------------------------------

